# Inline Profiler

Inline Profiler is designed to make better profiling experience than NsightSystem in some aspect and focusing on specific important modules, finally let developer know whether the optimization is really scaling.

Currently NsightSystem has several disadvatanges when profiling HugeCTR:

1. It introduce some overheads.
2. There are many blanks in timeline view and too many kernels. It makes hard for developers to have a clear vision on things they are interested.

So, we come up with this inline profiler, it:

1. Try to reduce overhead as much as possible by running profiling on only 1 kernel per iteration with CudaEvent.
2. Let user define what they are interested and try to provide a more clear vision on how things are optimized.
3. Introduce zero overhead for the code in the formal release version.

## Usage

1. Add `-DENABLE_PROFILING=ON` when pass args to Cmake. And then do the normal `make` process.

2. In `tools/profiler/schedule.py`, there defines all the optional events you may want to profile in each layer. You can just comment out some events you are not inerested in the `dlrm_perf_schedule`. If you want to add new event, please contact Randy Wang or Daniel Abel. It would be better to insert the profile macro `PROFILE_RECORD(label, stream, device_id)` by the profiler developer. We will try to meet developers' need and add necessary events.

3. `cd HugeCTR_PROJECT_ROOT` and `python3 tools/profiler/schedule.py`. This will generate a `prof.schedule` file under the project root. It is used to instruct profiler to schedule profiling.

4. Then just train the hugectr like normal, for example `NCCL_LAUNCH_MODE=PARALLEL ./build//bin/huge_ctr --train ./experiment/dlrm_fp16_16k.json`. The profiler will collect data during training and prof only 1 kernel per iteration and the program will terminate after profiling complete.

5. A file named 'prof.json' will appear in the project root. It records the kernel execution order, stream, device, prof time, etc.


In [ ]:
%matplotlib notebook
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 1000px; }</style>"))
display(HTML("<style>pre { white-space: pre !important; }</style>"))

import sys
import os
from datetime import datetime
from shutil import copy

from IPython.display import display

sys.path.append(os.path.join(os.path.abspath(''), '..', '..'))

from tools.profiler import generate_schedule, parse_result, print_result, timeline_chart

dlrm_perf_schedule = {
    # interested event name
    'BottomMLP.fc1': {
        'forward_events': [
            'fused_relu_bias_fully_connected.fprop',
            'fused_relu_bias_fully_connected.fprop.cublasLtMatmul',
        ],
        'backward_events': [
            'fused_relu_bias_fully_connected.bprop',
            'fused_relu_bias_fully_connected.bprop.initialize_array',
            'fused_relu_bias_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_relu_bias_fully_connected.bprop.convert_array',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_1',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'BottomMLP.fc2': {
        'forward_events': [
            'fused_relu_bias_fully_connected.fprop',
            'fused_relu_bias_fully_connected.fprop.cublasLtMatmul',
        ],
        'backward_events': [
            'fused_relu_bias_fully_connected.bprop',
            'fused_relu_bias_fully_connected.bprop.initialize_array',
            'fused_relu_bias_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_relu_bias_fully_connected.bprop.convert_array',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_1',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'BottomMLP.fc3': {
        'forward_events': [
            'fused_relu_bias_fully_connected.fprop',
            'fused_relu_bias_fully_connected.fprop.cublasLtMatmul',
        ],
        'backward_events': [
            'fused_relu_bias_fully_connected.bprop',
            'fused_relu_bias_fully_connected.bprop.initialize_array',
            'fused_relu_bias_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_relu_bias_fully_connected.bprop.convert_array',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_1',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'sparse_embedding1': {
        'forward_events': [
            'localized_slot_sparse_embedding_one_hot.forward.mapping_and_fuse'
        ],
        'backward_events': []
    },
    'interaction1': {
        'forward_events': [],
        'backward_events': [] 
    },
    'TopMLP.fc4': {
        'forward_events': [
            'fused_relu_bias_fully_connected.fprop',
            'fused_relu_bias_fully_connected.fprop.cublasLtMatmul',
        ],
        'backward_events': [
            'fused_relu_bias_fully_connected.bprop',
            'fused_relu_bias_fully_connected.bprop.initialize_array',
            'fused_relu_bias_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_relu_bias_fully_connected.bprop.convert_array',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_1',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'TopMLP.fc5': {
        'forward_events': [
            'fused_relu_bias_fully_connected.fprop',
            'fused_relu_bias_fully_connected.fprop.cublasLtMatmul',
        ],
        'backward_events': [
            'fused_relu_bias_fully_connected.bprop',
            'fused_relu_bias_fully_connected.bprop.initialize_array',
            'fused_relu_bias_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_relu_bias_fully_connected.bprop.convert_array',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_1',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'TopMLP.fc6': {
        'forward_events': [
            'fused_relu_bias_fully_connected.fprop',
            'fused_relu_bias_fully_connected.fprop.cublasLtMatmul',
        ],
        'backward_events': [
            'fused_relu_bias_fully_connected.bprop',
            'fused_relu_bias_fully_connected.bprop.initialize_array',
            'fused_relu_bias_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_relu_bias_fully_connected.bprop.convert_array',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_1',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'TopMLP.fc7': {
        'forward_events': [
            'fused_relu_bias_fully_connected.fprop',
            'fused_relu_bias_fully_connected.fprop.cublasLtMatmul',
        ],
        'backward_events': [
            'fused_relu_bias_fully_connected.bprop',
            'fused_relu_bias_fully_connected.bprop.initialize_array',
            'fused_relu_bias_fully_connected.bprop.reverse_add_bias_and_re_kernel',
            'fused_relu_bias_fully_connected.bprop.convert_array',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_1',
            'fused_relu_bias_fully_connected.bprop.cublasGemmEx_2'            
        ]
    },
    'TopMLP.fc8': {
        'forward_events': [],
        'backward_events': []
    },
}

In [ ]:
working_dir = os.path.abspath('')
# some configs
slurm_nodes = 4
slurm_account = "mlperft-dlrm"
container_image = "gitlab-master.nvidia.com/dl/hugectr/hugectr:mlperf_multi-node_ib_nightly_ci.latest"
container_name = "inline_profiler_hugectr"
container_mounts="/lustre/fsr:/raid,/lustre/fsw/mlperft-dlrm/ruotongw/hugectr:/etc/workspace/home"
config_file = os.path.join(working_dir, '../../mlperf/configs/55296_8gpus.json')

profiling_dir = os.path.join(working_dir, 'test')
#profiling_dir = os.path.join(working_dir, datetime.now().strftime("%Y-%d%m-%H-%M-%S"))
os.makedirs(profiling_dir, exist_ok=True)
copy(config_file, profiling_dir)
# generate the schedule
generate_schedule(dlrm_perf_schedule, profiling_dir)

In [ ]:
# run the training
!srun \
    -A mlperft-dlrm \
    --container-image="gitlab-master.nvidia.com/dl/hugectr/hugectr:mlperf_multi-node_ib_nightly_ci.latest" \
    --container-name=inline_profiler_hugectr \
    --container-workdir=/etc/workspace/home \
    --container-mounts=/lustre/fsr:/raid,/lustre/fsw/mlperft-dlrm/ruotongw/hugectr:/etc/workspace/home \
    --ntasks=2 \
    --ntasks-per-node 1 \
    --export=EDITOR,NCCL_LAUNCH_MODE=PARALLEL,PROFILING_DIR=tools/profiler/2021-0102-14-12-15 \
    build/bin/huge_ctr --train mlperf/configs/55296_8gpus.json

In [ ]:
result = parse_result(profiling_dir)
print_result(result)

In [ ]:
# Draw a timeline chart
timeline_chart(result)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#profiling_dirs = ['dgxA100-1node-batch55k', 'dgxA100-2node-batch55k', 'dgxA100-4node-batch55k']
profiling_dirs = [profiling_dir]
results = []
names = []
for prof_dir in profiling_dirs:
    names.append(prof_dir)
    results.append(parse_result(prof_dir))

def scaling_chart(results, names):

    bar_width = 0.2
    x_label_font_size = 7
    names_font_size = 9
    fig, ax = plt.subplots()
    
    iter_time_data = []
    label_time_avg_data = []
    label_time_max_data = []
    
    x_axis = np.array([i + 1 for i in range(len(results))])
    for result in results:
        iter_times = [host["avg_iter_time_ms"] for host in result]
        avg_iter_times = sum(iter_times) / len(iter_times)
        iter_time_data.append(avg_iter_times)
        
        labels = {}
        for host in result:
            for device in host["timeline"].keys():
                for stream in host["timeline"][device].keys():
                    for event in host["timeline"][device][stream]:
                        if event['label'] not in labels.keys():
                            labels[event['label']] = []
                        labels[event['label']].append(event['avg_measured_time_ms'])

        label_time_avg_data_this_result = []
        label_time_max_data_this_result = []
        for _, times in labels.items():
            label_time_avg_data_this_result.append(sum(times) / len(times))
            label_time_max_data_this_result.append(max(times))
        label_time_avg_data.append(label_time_avg_data_this_result)
        label_time_max_data.append(label_time_max_data_this_result)

    iter_time_data = np.array(iter_time_data)
    label_time_avg_data = np.array(label_time_avg_data)
    label_time_max_data = np.array(label_time_max_data)
    
    plt.bar(x_axis, iter_time_data, bar_width, align='edge')
    accumulate_label_time_avg_data = np.zeros_like(label_time_avg_data[:, 0])
    accumulate_label_time_max_data = np.zeros_like(label_time_max_data[:, 0])
    
    for i in range(label_time_avg_data.shape[1]):
        plt.bar(x_axis + bar_width, label_time_avg_data[:, i], bar_width,
                bottom=accumulate_label_time_avg_data, align='edge')
        plt.bar(x_axis + 2 * bar_width, label_time_max_data[:, i], bar_width,
                bottom=accumulate_label_time_max_data, align='edge')
        accumulate_label_time_avg_data += label_time_avg_data[:, i]
        accumulate_label_time_max_data += label_time_max_data[:, i]
        
    x_ticks = np.concatenate((x_axis + bar_width / 2, x_axis + bar_width + bar_width / 2,
                             x_axis + 2 * bar_width + bar_width / 2))
    x_labels = ['Whole iteration', 'Avg_mesured_ms', 'Max_measured_ms'] * len(results)
    print(x_ticks)
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(x_labels, fontdict={ 'fontsize' : x_label_font_size })
    
    for i in range(len(results)):
        annot = ax.annotate(names[i], xy=(i + 1 + 3 * bar_width / 2, 0), xytext=(0, -30), 
                            textcoords="offset points", fontsize=names_font_size, ha='center', va='center')

    ax.grid(True)
    fig.set_size_inches(8, 10)
    plt.show()
    
scaling_chart(results, names)